In [19]:
import os

base_dir = 'G:/IAMMMM/data'
all_images = []

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for f in os.listdir(folder_path):
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
                all_images.append(os.path.join(folder_path, f))

print(f'Tổng số ảnh tìm thấy: {len(all_images)}')


Tổng số ảnh tìm thấy: 1539


In [10]:
import os
import cv2
import numpy as np
from tqdm import tqdm
def preprocess_image(image_path, output_dir):
    # Đọc ảnh
    img = cv2.imread(image_path)
    if img is None:
        return
    
    # 1. Chuyển grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 2. Cân bằng sáng (CLAHE để tránh over-amplification noise)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized = clahe.apply(gray)
    
    # 3. Lọc nhiễu (Median Blur cho ảnh có nhiễu salt & pepper)
    denoised = cv2.medianBlur(equalized, 3)
    
    # 4. Binarization (Adaptive Thresholding cho ảnh ánh sáng không đều)
    binary = cv2.adaptiveThreshold(
        denoised, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # 5. Lưu ảnh đã xử lý
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, binary)

# Thư mục chứa ảnh gốc và thư mục output
input_dir = 'G:/IAMMMM/data'
output_dir = 'E:/IAMMMM/processed_data'

# Tạo thư mục output nếu chưa có
os.makedirs(output_dir, exist_ok=True)

# Tiền xử lý toàn bộ ảnh
for folder in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, folder)
    if os.path.isdir(folder_path):
        for f in tqdm(os.listdir(folder_path)):
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
                preprocess_image(
                    os.path.join(folder_path, f), 
                    output_dir
                )

100%|██████████| 10/10 [00:01<00:00,  6.84it/s]


In [7]:
import os

print(os.getcwd())
output_dir = 'E:/IAMMMM/processed_data'# Current working directory
print(output_dir) 

G:\Compvision Final\pythonProject
E:/IAMMMM/processed_data


In [13]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Đường dẫn gốc
base_dir = r"E:\IAMMMM\processed_data"
output_train_dir = os.path.join(base_dir, "train")
output_test_dir = os.path.join(base_dir, "test")

# Tạo thư mục train/test nếu chưa có
os.makedirs(output_train_dir, exist_ok=True)
os.makedirs(output_test_dir, exist_ok=True)

# Lấy danh sách ảnh đã xử lý
processed_images = [
    os.path.join(base_dir, f) 
    for f in os.listdir(base_dir) 
    if f.endswith('.png')
]

# Chia train/test 80/20
train_images, test_images = train_test_split(
    processed_images, test_size=0.2, random_state=42
)

# Copy ảnh vào từng thư mục
for img in train_images:
    shutil.copy(img, os.path.join(output_train_dir, os.path.basename(img)))

for img in test_images:
    shutil.copy(img, os.path.join(output_test_dir, os.path.basename(img)))

print(f"Số ảnh train: {len(train_images)}")
print(f"Số ảnh test: {len(test_images)}")


Số ảnh train: 1231
Số ảnh test: 308
